# Feedforward Networks for Handwritten Digit Recognition

In this assignment you will learn how to use feedforward neural networks to solve a classical task in machine learning: handwritten digit recognition using images from the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). More concretely, you will have to solve the following tasks:

1. implement a deep feedforward network that reads a batch of images and predicts the corresponding digits
2. train this network using stochastic gradient descent

In order to run the code for this assignment, you need a working installation of [NumPy](http://www.numpy.org). Check whether everything works by running the following code cell:

In [ ]:
import numpy as np

## Data

The MNIST dataset is split into a training set with 60,000 instances and a test set with 10,000 instances. Each instance consists of a greyscale image of a handwritten digit and an integer representing the digit in the image, as labelled by human experts. The digits are scaled and centred on a 28-by-28 pixel canvas.

The following code will read the training data and the test data:

In [ ]:
import mnist_network

training_x, training_y = mnist_network.read_training_data()
print('Shapes of the training data matrices: {} {}'.format(training_x.shape, training_y.shape))

test_x, test_y = mnist_network.read_test_data()
print('Shapes of the test data matrices: {} {}'.format(test_x.shape, test_y.shape))

Shapes of the training data matrices: (60000, 784) (60000, 10)
Shapes of the test data matrices: (10000, 784) (10000, 10)


From a Python perspective, each of the two data sets is a pair `(x, y)` of matrices: Each row of `x` is a 784-component vector containing the greyscale values of the pixels in an image as floats between 0 and 1. Each row of `y` is a 10-component one-hot vector representing the digit corresponding to the image. As an example, here is the vector for the first digit in the test data, the digit 7:


In [ ]:
test_y[1]


array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

## Task 1: Implement the network

Your first task is to implement a deep feedforward network that reads a batch of image vectors and predicts the corresponding digits. Your network should conform to the following specification:

* one input layer, one output layer, flexible number of hidden layers
* activation function for hidden layers: rectified linear unit (ReLU)
* activation function for output layer: softmax
* error function for gradient computation: categorical cross-entropy

To get you started on this task, we provide skeleton code and a number of useful helper functions.

### Helper functions

The following cell contains NumPy-based implementations of the ReLU activation function and its derivative (which you should use for the hidden layers of your network), as well as the softmax activation function (for the output layer).

In [ ]:
def relu(x):
    return x * (x > 0)

def relu_prime(x):
    return 1 * (x > 0)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return e_x / e_x.sum(axis=1, keepdims=True)

In each case, the argument `x` is a batch of input values, such as `training_x`. The implementation of the softmax function uses a standard trick to improve numerical stability; see [this link](http://stackoverflow.com/questions/34968722/softmax-function-python) for more information.

### Skeleton code

To get you started, we provide the following skeleton code:

In [ ]:
class Network():

    def __init__(self, sizes):
        self.sizes = sizes
        self.ws = [np.random.randn(m, n)*0.01 for m, n in zip(self.sizes[:-1], self.sizes[1:])] 
        self.bs = [np.random.randn(1, n)*0 for n in self.sizes[1:]]

    # forward: computes the output of the network for a batch of input values
    def forward(self, x):
        hidden_layers_activation_functions = list()
        for w,b in zip(self.ws[:-1],self.bs[:-1]):
            hidden_layer_activation= relu(np.dot(x, w) + b) 
            x = hidden_layer_activation
            hidden_layers_activation_functions.append(hidden_layer_activation)
        # hidden_layers is a list of activation functions for the different layers
        output = softmax(np.dot(hidden_layers_activation_functions[-1], self.ws[-1]) + self.bs[-1]) 
        return output

    # pick the digit with the highest probability
    def predict(self, x):
        return np.argmax(self.forward(x), axis=1)


    # backpropagate: computes the network gradients for a batch of input and corresponding output values
    def backpropagate(self, x, y):
        inputs = x
        forward_probabilities = self.forward(np.asarray(x))                     
        delta_output =  forward_probabilities - y
        # hidden layers' activation functions plus the inputs x
        hidden_layers_activation_functions = list()
        hidden_layers_activation_functions.append(inputs)
        for w,b in zip(self.ws[:-1],self.bs[:-1]):
            hidden_layer_activation= relu(np.dot(x, w) + b) 
            x = hidden_layer_activation
            hidden_layers_activation_functions.append(hidden_layer_activation)
        #output = softmax(np.dot(hidden_layers_activation_functions[-1], self.ws[-1]) + self.bs[-1]) 
        #hidden_layers_activation_functions.append(output)
        grad_out = np.dot(delta_output.T, hidden_layers_activation_functions[-1]).T
        grad_bias_out = np.sum(delta_output, axis=0, keepdims=True)
        # delta_layers is a list of delta values for the hidden layers and the output layer
        delta_layers = list()
        delta_layers.append(delta_output)
        grad_bias_layers = list()
        grad_layers = list()
        weights = self.ws.copy()

        for w,b in zip(reversed(self.ws[:-1]), reversed(self.bs[:-1])):
            relu_prime_factor = relu_prime(np.dot(hidden_layers_activation_functions[-2], w) + b) 
            sum_product = np.dot(delta_layers[-1], weights[-1].T)                                   
            delta_layer = np.multiply(relu_prime_factor, sum_product)

            delta_layers.append(delta_layer)

            grad_bias_layer = np.sum(delta_layer, axis=0, keepdims=True)
            grad_bias_layers.append(grad_bias_layer)
            
            x = hidden_layers_activation_functions[-2]

            grad_layer = np.dot(x.T, delta_layer)
            grad_layers.append(grad_layer)

            weights.pop(-1)
            hidden_layers_activation_functions.pop(-1)
             
        grad_layers.reverse()
        grad_layers.append(grad_out)
        grad_bias_layers.reverse()
        grad_bias_layers.append(grad_bias_out)

        return grad_layers, grad_bias_layers 


This code defines a class Network that represents deep feedforward networks. The intended behaviour of the fields and methods of this class is specified below. In the skeleton code, all fields are initialized with, and all methods return, zero matrices of the appropriate shape. In order to obtain a network that meets the requirements, you will have to write code that replaces these placeholders with meaningful values.

In your implementation, you may choose to add more fields and/or methods than the ones included in the skeleton code. However, in all of your code, you may only call functions from the NumPy library, but no other library.

### Fields

**sizes : list(int)**

The dimensions of the network layers, from the first (input) layer to the last (output) layer.

An example, in a network with 784 units in the input layer, 10 units in the output layer, and 100 units in the (single) hidden layer this field would have the value

In [ ]:
[784, 100, 10]

[784, 100, 10]

**ws : list(np.array)**

The weight matrices of the network, where the matrix at index $i$ holds the weights of the connections from layer $i$ to layer $i+1$. As an example, if the shape of layer&nbsp;0 is $(784, 100)$ and the shape of layer&nbsp;1 is $(100, 10)$, then `ws[0]` will have shape $(100, 10)$.

**bs : list(np.array)**

The bias vectors of the network, where the vector at index $i$ holds the biases for layer $i+1$. As an example, `bs[0]` is the bias vector of layer&nbsp;1. Note that there are no biases for the input layer (layer&nbsp;0).

### Initialization

Initialize the weights and biases of the network. Note that in the starter code, both weights and biases are initialized using zeros.

**sizes : list(int)**

The dimensions of the network layers, from the first (input) layer to the last (output) layer.

As an example, the following code creates a network with 784 units in the input layer, 10 units in the output layer, and 100 units in the (single) hidden layer:

In [ ]:
net = Network([784, 100, 10])

### forward

Computes the output of the network for a batch of input values.

**x : np.array**

A batch of input values, such as `training_x`.

**Returns:** The output of the network for the specified input. This will be an array of shape $(m, n)$ where $m$ is the number of rows in the input batch, and $n$ is the size of the last layer in the network. In the starter code, the method returns an array of all zeros.

### predict

Predicts the digits for a batch of input values.

**x : np.array**

A batch of input values, such as `test_x`.

**Returns:** The digits predicted for the specified input. This will be an array of shape $(m, 1)$ where $m$ is the number of rows in the input batch $x$. In the starter code, the method returns an array of all zeros.

### backpropagate

Computes the network gradients for a batch of input and corresponding output values. Note that in the context of this assignment, the gradients should be computed relative to categorical cross-entropy as the error function.

**x : np.array**

A batch of input values, such as `training_x`.

**y : np.array**

A batch of corresponding output values, such as `training_y`.

**Returns:** A list of pairs of the form $(\nabla w, \nabla b)$, one for each non-input layer of the network, where the first component of each pair is the average gradient for the weights of the connections coming into the layer and the second component is the average gradient for the biases at the layer. In the starter code, the method returns a list of zero gradients.

## Task 2: Train your network

Once you have completed the Network class, your second task is to write code to train the network using stochastic gradient descent (SGD).

### Helper functions

The function in the next code cell will sample minibatches from an array `x` of input values and a corresponding array `y` of output values:

In [ ]:
def minibatches(x, y, batch_size):
    random_indices = np.random.permutation(np.arange(x.shape[0]))
    for i in range(0, x.shape[0] - batch_size + 1, batch_size):
        batch_indices = random_indices[i:i+batch_size]
        yield x[batch_indices], y[batch_indices]

The next function computes the test error rate of a network on a batch of test data.

In [ ]:
def evaluate(net):
    return np.mean(net.predict(test_x) != np.argmax(test_y, axis=1))

### Skeleton code

The following cell contains skeleton code for the training algorithm.

In [ ]:
# train_sgd: the training algorithm
def train_sgd(net, x, y, n_epochs, batch_size, eta=0.007):
    for t in range(n_epochs):
        for batch_x, batch_y in minibatches(x, y, batch_size):
            grad_layers, grad_bias_layers = net.backpropagate(batch_x,batch_y)
            for i in range(len(grad_layers)):
                net.ws[i] = net.ws[i] - np.dot(eta, grad_layers[i])
                net.bs[i] = net.bs[i] - np.dot(eta, grad_bias_layers[i])
        print("epoch = {}, test error rate = {:.4f}".format(t, evaluate(net)))


The intended meaning of the various parameters is as follows:

**x : np.array**

A batch of input values, such as `training_x`.

**y : np.array**

A batch of corresponding output values, such as `training_y`.

**n_epochs : int**

The number of iterations over the training data (&lsquo;epochs&rsquo;).

**batch_size : int**

The number of input values per minibatch.

**eta : float**

The learning rate in the stochastic gradient descent update step.

### Intended usage

To see how the training code is intended to be used, here is how you set up a network and train it on the training data for 2&nbsp;iterations with minibatch size&nbsp;30 and the default learning rate.

In [ ]:
net = Network([784, 300, 120, 10])
train_sgd(net, training_x, training_y, 10, 30)

epoch = 0, test error rate = 0.0402
epoch = 1, test error rate = 0.0317
epoch = 2, test error rate = 0.0279
epoch = 3, test error rate = 0.0230
epoch = 4, test error rate = 0.0223
epoch = 5, test error rate = 0.0197
epoch = 6, test error rate = 0.0209
epoch = 7, test error rate = 0.0169
epoch = 8, test error rate = 0.0183
epoch = 9, test error rate = 0.0192


### Performance goal

Once you have a working network and training algorithm, you can compare the error rate of your network to the results on the [MNIST website](http://yann.lecun.com/exdb/mnist/).

**To get credit for this assignment, your network must achieve a test error rate of less than 2% at least once during the first 10 epochs of training.**

To tune your network, you can play around with the various training parameters: number of epochs, minibatch size, and learning rate. In addition to that, you can also make more substantial changes such as the following:

* Make the network wider (increase the size of a layer) or deeper (add more layers).
* Implement a different initialization strategy.
* Implement a regularization method or dropout.
* Implement an optimization algorithm with an adaptive learning rate, such as RMSProp or Adam.

## How to submit

When you have reached the performance goal, send this notebook to Marco to receive credit for the assignment. The notebook must be self-contained and must run without error.

In addition to your code, you are asked to submit a short text (less than 500&nbsp;words) in which you reflect on what you have done. Which specific choices did you make when tuning your network? How did these choices affect performance? You can enter your text in the cell below.

### Report

In this lab, a neural network with an input layer and three hidden layers was created. In order to get less than 2% error rate, tuning was necessary. The learning rate should not be too large or too small. The weights and bias initialization is very important and different approaches should be tested in order to get the small error rate. The minibatch size is kept small.

*Good luck!*